<a href="https://colab.research.google.com/github/NoamMichael/Comparing-Confidence-in-LLMs/blob/main/BoolQ_Benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Notebook for benchmarking BoolQ




In [2]:
%pip install anthropic
%pip install openai
%pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 6.0 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import json
import time
import random
import torch
import matplotlib.pyplot as plt
from transformers import (AutoTokenizer,
                        AutoModelForCausalLM,
                        BitsAndBytesConfig,
                        pipeline)
import warnings
import openai
import anthropic
import google.generativeai as genai
from abc import ABC, abstractmethod
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
from google.colab import userdata

max_tokens = 150

class OpenModel: ## This class is built around Hugging Face methods
  def __init__(self, name, key, MaxTokens = 150):
    self.name = name
    self.key = key
    self.MaxTokens = MaxTokens
    print(f"Downloading Tokenizer for {self.name}")
    self.tokenizer = AutoTokenizer.from_pretrained(self.name,token = self.key) ## Import Tokenizer
    print(f"Downloading Model Weights for {self.name}")
    self.model = AutoModelForCausalLM.from_pretrained(self.name, token = self.key, device_map="auto") ## Import Model

    ## Make text generation pipeline
    self.pipeline = pipeline(
    "text-generation",
    model = self.model,
    tokenizer = self.tokenizer,
    do_sample = False,
    max_new_tokens = self.MaxTokens,
    eos_token_id = self.tokenizer.eos_token_id,
    pad_token_id = self.tokenizer.eos_token_id
    )

  def generate(self, prompt):
    return self.pipeline(prompt)[0]['generated_text']

  def GetTokens(self, prompt: str):
    ## Get Answer:
    batch = self.tokenizer(prompt, return_tensors= "pt").to('cuda')
    with torch.no_grad():
        outputs = self.model(**batch)
    ## Get Token Probabilites
    logits = outputs.logits

    ## Apply softmax to the logits to get probabilities
    probs = torch.softmax(logits[0, -1], dim=0)

    ##Get the top k token indices and their probabilities
    top_k_probs, top_k_indices = torch.topk(probs, 100, sorted =True)

    ## Convert token indices to tokens
    top_k_tokens = [self.tokenizer.decode([token_id]) for token_id in top_k_indices]

    ## Convert probabilities to list of floats
    top_k_probs = top_k_probs.tolist()                  #list of probabilities

    ## Create a Pandas Series with tokens as index and probabilities as values
    logit_series = pd.Series(top_k_probs, index=top_k_tokens)

    ## Sort the series by values in descending order
    logit_series = logit_series.sort_values(ascending=False)
    logit_series.index.name = "Token"
    logit_series.name = "Probability"
    return logit_series

class ClosedModel(ABC):
  @abstractmethod
  def generate(self, prompt: str, system:str = "")-> str:
        """
        Abstract method to generate a response from the language model.
        """
        pass
  @abstractmethod
  def __init__(self, name, api_key):
    self.name = name
    self.key = api_key
    self.results = pd.DataFrame(columns = ['Question ID','Question', 'Answer', 'Reasoning', 'Error'])
    pass

  @abstractmethod
  def client(self):
    pass

  @abstractmethod
  def GetRAC(self, prompt: str, system1:str = "", system2: str = "")-> str: ## Get Reasoning Answer Confidence
    pass

class GPTmodel(ClosedModel):
  def __init__(self, name, api_key):
    self.name = name
    self.key = api_key
    self.results = pd.DataFrame(columns = ['Question ID','Question', 'Answer', 'Error'])
  def client(self):
    # Initialize the OpenAI client with the API key
    self.client = openai.OpenAI(api_key=self.key)


  def generate(self, prompt: str, system: str = "") -> str:
    # Use the new client-based API call
    response = self.client.chat.completions.create(
        model=self.name,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens= max_tokens
    )
    # Access the content from the new response object structure
    return response.choices[0].message.content


  def GetRAC(self, prompt: str)-> str: ## Get Reasoning Answer Confidence
    # Access global system prompts
    global sys_prompt1
    ## Get the reasoning
    rac = self.generate(prompt, sys_prompt1)
    ## Get the answer and confidence
    return rac

class AnthropicModel(ClosedModel):
  def __init__(self, name, api_key):
    self.name = name
    self.key = api_key
    self.results = pd.DataFrame(columns = ['Question ID','Question', 'Answer', 'Error'])
  def client(self):
    # Initialize the Anthropic client with the API key
    self.client = anthropic.Anthropic(api_key=self.key)

  def generate(self, prompt: str, system: str = "") -> str:
    # The messages list should only contain user and assistant roles
    messages = [{"role": "user", "content": prompt}]

    # Use the Anthropic client to create a message
    # Pass the system message as a top-level 'system' parameter
    message = self.client.messages.create(
        model=self.name,
        max_tokens= max_tokens, # You can adjust this or make it an instance variable
        messages=messages,
        system=system if system else None # Pass system as a separate parameter, or None if empty
    )
    # Access the content from the response object
    return message.content[0].text

  def GetRAC(self, prompt: str)-> str: ## Get Reasoning Answer Confidence
    # Access global system prompts
    global sys_prompt1
    ## Get the reasoning
    rac = self.generate(prompt, sys_prompt1)
    ## Get the answer and confidence
    return rac


class GeminiModel(ClosedModel):
  def __init__(self, name, api_key):
    self.name = name
    self.key = api_key
    self.results = pd.DataFrame(columns = ['Question ID','Question', 'Answer', 'Error'])

  def client(self):
    # Initialize the google.generativeai client with the API key

    genai.configure(api_key=self.key)
    self.model = genai.GenerativeModel(model_name=self.name)

  def generate(self, prompt: str, system: str = "") -> str:
    # Build the content list, including the system message if provided
    contents = [{"role": "user", "parts": [prompt]}]
    if system:
        contents = [{"role": "user", "parts": [system]}] + contents

    # Use the Gemini model to generate content
    response = self.model.generate_content(contents)

    # Access the content from the response object
    return response.text

  def GetRAC(self, prompt: str)-> str: ## Get Reasoning Answer Confidence
    # Access global system prompts
    global sys_prompt1
    ## Get the reasoning
    rac = self.generate(prompt, sys_prompt1)
    ## Get the answer and confidence
    return rac

In [24]:
## Define Functions:
def init_models(models_dict,
                test_prompt = "Zdzisław Beksiński was",
                test_system = "You are a helpful assistant."):
  print('Initializing Closed Models:')
  closed_models = []
  for model_type in my_closed_models:
      print(f'{model_type}:')
      api_key_name = my_closed_models[model_type]['api_key_name']
      api_key = userdata.get(api_key_name)
      print(f'  API Key Name: {my_closed_models[model_type]["api_key_name"]}')
      for model_name in my_closed_models[model_type]['models']:
        # Instantiate the correct subclass based on model_type
        if model_type == 'GPT':
            my_model = GPTmodel(name = model_name, api_key = api_key)
        elif model_type == 'Claude':
            my_model = AnthropicModel(name = model_name, api_key = api_key)
        elif model_type == 'Gemini':
            my_model = GeminiModel(name = model_name, api_key = api_key)
        else:
            # Handle unexpected model types if necessary
            print(f"Warning: Unknown model type {model_type}. Skipping.")
            continue # Skip to the next model name if type is unknow
        my_model.client()
        closed_models.append(my_model)
        print(f'    {model_name}')


  print(f'Models Initialized: {len(closed_models)}')
  print(f'Model locations:\n{closed_models}')

  print('-'*42)
  print('Testing all closed models:')
  print(f'Test prompt: {test_prompt}')
  print(f'Test system: {test_system}')

  for model in closed_models:
    print(f'\nTesting model: {model.name}')
    print(model.generate(test_prompt, test_system))
  return closed_models

def format_df(df):

  ## %%%%%%%%%%%%%%
  ## I need to fix how formating is done for some Q's. As daniel pointed out some
  ## questions only have 4 options, not 5.
  ## %%%%%%%%%%%%%%

  ## Takes in a dataframe in the form:
  ## | Question Number | Question | Option A | Option B | ... | Correct Answer Letter |
  ## |     (Int)       |     (Str)     |  (Str)   |  (Str)   |     |       (Char)          |
  ##
  ## Returns a dataframe in the form:
  ## | Question Number | Full Prompt 1 | Full Prompt 2 |
  ## |     (Int)       |    (Str)      |    (Str)      |

  columns = df.columns
  num_options = columns.str.contains('Option').astype(int).sum()

  #----------------------------------------------------------------------------#
  ## Check if DF is formatted properly
  error_text = f'''Make sure dataframe is in following format:
  | Question Number | Question | Option A | Option B | ... | Correct Answer Letter |
  |     (Int)       |     (Str)     |  (Str)   |  (Str)   |     |       (Char)          |

  The current format of Dataframe is: {columns}
  '''
  ['Question Number', 'Question', 'Correct Answer Letter']
  if num_options < 2:
    raise Exception(error_text)

  #----------------------------------------------------------------------------#
  ## Initialize Output dataframe:
  header = ['Question Num', 'Full Prompt 1', 'Full Prompt 2']
  output_df = pd.DataFrame(columns = header)

  #----------------------------------------------------------------------------#

  ## Format questions for benchmark
  letters = ['A', 'B', 'C', 'D', 'E']
  options = ['Option A', 'Option B', 'Option C', 'Option D', 'Option E']

  for i in range(len(df)):
    question = df['Question'][i]

    sys_prompt_temp1 = sys_prompt1
    sys_prompt_temp2 = sys_prompt2
    ## Reformat system prompt in order to fit number of options in benchmark
    if type(df['Option E'][i]) == float: ## ABCD
      sys_prompt_temp1 = (sys_prompt1
                    .replace('(A, B, C, D, or E)', '(A, B, C, or D)')
                    .replace('E) ${Option E}', '')
          )
      sys_prompt_temp2 = (sys_prompt2
                    .replace('(A, B, C, D, or E)', '(A, B, C, or D)')
                    .replace('E) ${Option E}', '')
          )
      if type(df['Option D'][i]) == float: ## ABC
        sys_prompt_temp1 = (sys_prompt_temp1
                      .replace('(A, B, C, or D)', '(A, B, or C)')
                      .replace('D) ${Option D}', '')
            )
        sys_prompt_temp2 = (sys_prompt_temp2
                    .replace('(A, B, C, or D)', '(A, B, or C)')
                    .replace('D) ${Option D}', '')
          )

        if type(df['Option C'][i]) == float: ## AB
          sys_prompt_temp1 = (sys_prompt_temp1
                        .replace('(A, B, or C)', '(A or B)')
                        .replace('C) ${Option C}', '')
              )
          sys_prompt_temp2 = (sys_prompt_temp2
                      .replace('(A, B, or C)', '(A or B)')
                      .replace('C) ${Option C}', '')
            )

    option_text = df[options[:num_options]].iloc[i].to_list()
    ## Prompt for specific question
    new_prompt = sys_prompt_temp1.replace('${Question}', question)
    for j in range(num_options): ## This for loop allows for dynamic question amounts
        new_prompt = new_prompt.replace(f'${{Option {letters[j]}}}', str(option_text[j]))


    ## Add formatted prompts.
    ## Note that this is formatted to llama so changes may be needed down the line.
    prompts1 = (new_prompt.split('<Your concise reasoning here. Max 100 words>')[0]) ## Specific prompt for question

    prompts2 = (sys_prompt_temp2) ## Generic prompt for question confidence
    output_df.loc[i] = [df['Question Number'].iloc[i], prompts1, prompts2]

  return output_df

def test_models_sequential_by_question(df, models, debug=False, start = 0):
    """
    Tests a list of models on a given dataset sequentially,
    iterating through questions and then models for each question.
    Includes a debug mode to process only the first 10 questions.

    Args:
        df (pd.DataFrame): The dataset containing questions and prompts.
        models (list): A list of initialized model objects.
        debug (bool): If True, only process the first 10 questions.
    """
    print("Clearing previous results for each model...")
    for model in models:
        model.results = pd.DataFrame(columns=['Question ID', 'Question', 'Answer', 'Error'])
        print(f"  Cleared results for {model.name}")
    print("Starting sequential testing (by question)...")

    # Determine the number of questions to process
    num_questions_to_process = 10 if debug else len(df)

    # Iterate over questions first
    for index, row in tqdm(df.iloc[start: num_questions_to_process].iterrows(), total=num_questions_to_process, desc="Processing Questions"):
        question_num = row['Question Number']
        prompt = row['Question'] + '\nResponse:\n'
        '''

        I dont love this implementation But honestly,
        unless my logic is flawed with some edge case, I think this should work
        and I dont want to rewrite the functions for each subclass to take in the
        dataframe in order to work.
        '''


        print(f"\nProcessing Question {question_num}")

        # Iterate over models for the current question
        for model in models:

            try:
                print(f"  Testing with model: {model.name}")
                # Call GetRAC and add the result to the model's self.results
                answer = model.GetRAC(prompt=prompt)

                # Add the results to the model's self.results DataFrame
                new_row = pd.DataFrame([{
                    'Question ID': question_num,
                    'Question': prompt,
                    'Answer': answer,
                    'Error': False
                }])
                model.results = model.results._append(new_row, ignore_index=True)
                filename = f"{model.name.replace('/', '_').replace('-', '_')}_test_results.csv"
                model.results.to_csv(filename, index=False)
            except Exception as e:
                print(f"  Error testing {model.name} on Question {question_num}: {e}")
                # Optionally add an error entry to the results
                error_row = pd.DataFrame([{
                    'Question ID': question_num,
                    'Question': prompt,
                    'Answer': f"Error: {e}",

                    'Error': True
                }])
                model.results = model.results._append(error_row, ignore_index=True)
                filename = f"{model.name.replace('/', '_').replace('-', '_')}_test_results.csv"
                model.results.to_csv(filename, index=False)
    print("\nSequential testing complete.")

    # After processing all questions, save the results for each model
    for model in models:
        filename = f"{model.name.replace('/', '_').replace('-', '_')}_test_results.csv"
        model.results.to_csv(filename, index=False)
        print(f"Results for {model.name} saved to '{filename}'")

In [26]:
## Import BoolQ
print('-' *42)
file_path = '/content/boolq_valid_formatted.csv'
print(f'Importing Dataset: {file_path}')
dataset = pd.read_csv(file_path)
dataset.head()

dataset['Question'].iloc[0] = 'does ethanol take more energy than it produces'
dataset['Question Number'] = dataset.index
display(dataset.head())


sys_prompt1 = '''
Analyze the following question and determine whether it is true or false. Your reasoning should not exceed 100 words. After your explanation, clearly state your answer.

Question: ${Question}

Resopnse: {
  'Reasoning': <Your concise reasoning here. Max 100 words>
  'Answer': <True or False>
  'Confidence': <Probability that your answer is correct. As a float from 0.0 to 1.0>
}

All options have a non-zero probability of being correct. No option should have a probability of 0 or 1.
Be modest about your certainty.  Do not provide any additional reasoning.
'''

## Initialize Models
print('-' *42)
print('Initializing Models:')
my_closed_models = {
    'GPT': {
        'api_key_name': 'gpt_api_key', # Name of the key to retrieve from userdata
        'models': [
            'gpt-4',
            'gpt-3.5-turbo'
        ]
    },
    'Claude': {
        'api_key_name': 'claude_api_key', # Name of the key to retrieve from userdata
        'models': [
            'claude-3-7-sonnet-20250219',
            'claude-3-haiku-20240307'
        ]
    },
    'Gemini': {
        'api_key_name': 'gemini_api_key', # Name of the key to retrieve from userdata
        'models': [
            'gemini-1.5-flash',
            'gemini-2.5-pro-preview-06-05'
        ]
    }
}

closed_models = init_models(my_closed_models)
print(' Successfully Initialied Models')




------------------------------------------
Importing Dataset: /content/boolq_valid_formatted.csv


,Question,Correct Answer,Question Number
0,does ethanol take more energy than it produces,False,0
1,is house tax and property tax are same,True,1
2,is pain experienced in a missing body part or ...,True,2
3,is harry potter and the escape from gringotts ...,True,3
4,is there a difference between hydroxyzine hcl ...,True,4


------------------------------------------
Initializing Models:
Initializing Closed Models:
GPT:
  API Key Name: gpt_api_key
    gpt-4
    gpt-3.5-turbo
Claude:
  API Key Name: claude_api_key
    claude-3-7-sonnet-20250219
    claude-3-haiku-20240307
Gemini:
  API Key Name: gemini_api_key
    gemini-1.5-flash
    gemini-2.5-pro-preview-06-05
Models Initialized: 6
Model locations:
[<__main__.GPTmodel object at 0x79c94d33c150>, <__main__.GPTmodel object at 0x79c94d529450>, <__main__.AnthropicModel object at 0x79c94f454350>, <__main__.AnthropicModel object at 0x79c94df55810>, <__main__.GeminiModel object at 0x79c94dfb3a50>, <__main__.GeminiModel object at 0x79c950e3a510>]
------------------------------------------
Testing all closed models:
Test prompt: Zdzisław Beksiński was
Test system: You are a helpful assistant.

Testing model: gpt-4
a renowned Polish painter, photographer, and sculptor. He is best known for his large, detailed images of a surreal, post-apocalyptic environment. Beksi

In [21]:
my_gpt = closed_models[0]

test_prompt = dataset['Question'].iloc[56]

print(my_gpt.GetRAC(test_prompt))



Response: {
  'Reasoning': Yes, there is a player in the NFL who is missing a hand. Shaquem Griffin, who was drafted by the Seattle Seahawks in 2018, was born with amniotic band syndrome which affected his left hand, leading to its amputation when he was four years old.
  'Answer': True
  'Confidence': 1.0
}


In [ ]:
test_models_sequential_by_question(dataset, closed_models)

Clearing previous results for each model...
  Cleared results for gpt-4
  Cleared results for gpt-3.5-turbo
  Cleared results for claude-3-7-sonnet-20250219
  Cleared results for claude-3-haiku-20240307
  Cleared results for gemini-1.5-flash
  Cleared results for gemini-2.5-pro-preview-06-05
Starting sequential testing (by question)...


Processing Questions:   0%|          | 0/3270 [00:00<?, ?it/s]


Processing Question 0
  Testing with model: gpt-4
  Testing with model: gpt-3.5-turbo
  Testing with model: claude-3-7-sonnet-20250219
  Testing with model: claude-3-haiku-20240307
  Testing with model: gemini-1.5-flash
  Testing with model: gemini-2.5-pro-preview-06-05

Processing Question 1
  Testing with model: gpt-4
  Testing with model: gpt-3.5-turbo
  Testing with model: claude-3-7-sonnet-20250219
  Testing with model: claude-3-haiku-20240307
  Testing with model: gemini-1.5-flash
  Testing with model: gemini-2.5-pro-preview-06-05

Processing Question 2
  Testing with model: gpt-4
  Testing with model: gpt-3.5-turbo
  Testing with model: claude-3-7-sonnet-20250219
  Testing with model: claude-3-haiku-20240307
  Testing with model: gemini-1.5-flash
  Testing with model: gemini-2.5-pro-preview-06-05

Processing Question 3
  Testing with model: gpt-4
  Testing with model: gpt-3.5-turbo
  Testing with model: claude-3-7-sonnet-20250219
  Testing with model: claude-3-haiku-20240307
  